# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the libraries required.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks  = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = "AAPL"
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2470818539357,
 'week52high': 158.84,
 'week52low': 116.4,
 'week52highSplitAdjustOnly': 159.1,
 'week52lowSplitAdjustOnly': 115.34,
 'week52change': 0.247583793812683,
 'sharesOutstanding': 17124242216,
 'float': 0,
 'avg10Volume': 61865711,
 'avg30Volume': 71339349,
 'day200MovingAvg': 144.43,
 'day50MovingAvg': 148.01,
 'employees': 150827,
 'ttmEPS': 11.79,
 'ttmDividendRate': 0.8738179346366027,
 'dividendYield': 0.006045611237461583,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-22',
 'nextEarningsDate': '2022-01-15',
 'peRatio': 13.276158765941256,
 'beta': 1.3785923991573534,
 'maxChangePercent': 57.33998265367482,
 'year5ChangePercent': 4.961277568513215,
 'year2ChangePercent': 1.3285395606244015,
 'year1ChangePercent': 0.2907685283511456,
 'ytdChangePercent': 0.12335308349879523,
 'month6ChangePercent': 0.17638362329257418,
 'month3ChangePercent': 0.0181561687571001,
 'month1ChangePercent': 0.03790997725834842,
 'day30ChangePe

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.2907685283511456

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
for batch in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={batch}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in batch.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A'
                    
                ],
                index = my_columns
            ),
            ignore_index=True
        )
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,160.390,0.469716,N/A
1,AAL,22.260,0.744944,N/A
2,AAP,243.210,0.532992,N/A
3,AAPL,153.248,0.291324,N/A
4,ABBV,118.460,0.245374,N/A
...,...,...,...,...
500,YUM,126.040,0.250258,N/A
501,ZBH,139.300,-0.10612,N/A
502,ZBRA,607.940,0.699983,N/A
503,ZION,67.580,0.727047,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.84,2.78517,N/A
1,275,LB,79.98,2.38865,N/A
2,317,MRO,17.27,2.36683,N/A
3,175,FANG,112.78,2.3024,N/A
4,195,FTNT,338.41,2.06873,N/A
5,299,MCHP,86.67,1.73696,N/A
6,357,OXY,33.43,1.62023,N/A
7,111,COTY,11.12,1.48091,N/A
8,345,NVDA,317.92,1.34096,N/A
9,174,F,20.13,1.33321,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio')

    try:
        portfolio_size = float(portfolio_size)
    except ValueError:
        print('That is not a number. \nPlease try again and enter a number.')
        portfolio_size = input('Enter the size of your portfolio')
        portfolio_size = float(portfolio_size)
portfolio_input()

In [9]:
position_size = portfolio_size/len(final_dataframe.index)
for i in range(0,len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

/Users/gyalpoaguirre/Dev/algorithmic_trading_python/env/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,148,DVN,43.84,2.78517,4562
1,275,LB,79.98,2.38865,2500
2,317,MRO,17.27,2.36683,11580
3,175,FANG,112.78,2.3024,1773
4,195,FTNT,338.41,2.06873,590
5,299,MCHP,86.67,1.73696,2307
6,357,OXY,33.43,1.62023,5982
7,111,COTY,11.12,1.48091,17985
8,345,NVDA,317.92,1.34096,629
9,174,F,20.13,1.33321,9935


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [41]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'One-Year Price Return',
    'One-Year Return Percentile', 
    'Six-Month Price Return',
    'Six-Month Return Percentile', 
    'Three-Month Price Return',
    'Three-Month Return Percentile', 
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score' 
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
for batch in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={batch}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in batch.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['price'],
                    'N/A',
                    data[symbol]['stats']['year1ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month6ChangePercent'],
                    'N/A',
                    data[symbol]['stats']['month3ChangePercent'],
                    'N/A', 
                    data[symbol]['stats']['month1ChangePercent'],
                    'N/A',
                    'N/A'
                ],
                index = hqm_columns
            ),
            ignore_index=True
        )
hqm_dataframe = hqm_dataframe.dropna()


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [42]:
time_periods = ['One-Year','Six-Month', 'Three-Month', 'One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.080,N/A,0.455873,68.8623,0.2179,85.2295,0.0189616,43.3134,0.0460644,50.0998,N/A
1,AAL,22.120,N/A,0.779163,89.8204,-0.0204949,31.1377,0.0309764,49.1018,0.0752228,68.6627,N/A
2,AAP,249.830,N/A,0.552051,77.4451,0.138728,70.8583,0.131397,82.8343,0.0974872,76.6467,N/A
3,AAPL,154.368,N/A,0.296539,46.1078,0.176673,79.0419,0.0174789,42.515,0.037565,45.1098,N/A
4,ABBV,117.760,N/A,0.245108,42.1158,0.0291378,46.507,0.0302606,48.1038,0.064345,62.6747,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [43]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,163.080,N/A,0.455873,68.8623,0.2179,85.2295,0.0189616,43.3134,0.0460644,50.0998,61.8762
1,AAL,22.120,N/A,0.779163,89.8204,-0.0204949,31.1377,0.0309764,49.1018,0.0752228,68.6627,59.6806
2,AAP,249.830,N/A,0.552051,77.4451,0.138728,70.8583,0.131397,82.8343,0.0974872,76.6467,76.9461
3,AAPL,154.368,N/A,0.296539,46.1078,0.176673,79.0419,0.0174789,42.515,0.037565,45.1098,53.1936
4,ABBV,117.760,N/A,0.245108,42.1158,0.0291378,46.507,0.0302606,48.1038,0.064345,62.6747,49.8503


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [44]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)
hqm_dataframe.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NVDA,315.74,N/A,1.34476,98.4032,1.11211,99.8004,0.481511,99.4012,0.428175,100,99.4012
1,F,19.64,N/A,1.33877,98.004,0.666839,98.6028,0.407588,99.002,0.288144,99.2016,98.7026
2,ANET,543.36,N/A,1.04754,95.4092,0.626343,98.004,0.411661,99.2016,0.405704,99.8004,98.1038
3,AMD,152.51,N/A,0.817438,91.018,0.876411,99.6008,0.323359,97.4052,0.346981,99.6008,96.9062
4,SPG,165.63,N/A,1.18786,96.6068,0.32625,93.8124,0.264779,96.6068,0.252689,98.2036,96.3074


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [45]:
portfolio_input()

In [46]:
position_size = portfolio_size/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i , 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [48]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [49]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [51]:
column_formats = {
    'A': ['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy', integer_template], 
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile',percent_template], 
    'F':['Six-Month Price Return', percent_template],
    'G':['Six-Month Return Percentile', percent_template], 
    'H':['Three-Month Price Return', percent_template],
    'I':['Three-Month Return Percentile', percent_template], 
    'J':['One-Month Price Return', percent_template],
    'K':['One-Month Return Percentile', percent_template],
    'L':['HQM Score', percent_template] 
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [52]:
writer.save()